In [2]:
import pandas as pd
from baseline_utils import get_data_for_NN, set_seed
import os

In [3]:
set_seed(420)

In [16]:
def compute_accuracy(y, yhat):
    correct = sum(1 for yt, yh in zip(y, yhat) if yt == yh)
    return correct / len(y)

def compute_recall(y, yhat):
    true_positives = sum(1 for yt, yh in zip(y, yhat) if yt == 1 and yh == 1)
    false_negatives = sum(1 for yt, yh in zip(y, yhat) if yt == 1 and yh == 0)
    # Avoid division by zero
    if true_positives + false_negatives == 0:
        return 0.0
    return true_positives / (true_positives + false_negatives)

# Label length weirdness
While comparing actual and predicted labels, we came across the weird problem, that the labels did not match the original text inputs anymore. To be exact, the labels were suddenly more numerous than the tokens we had them for. Here is the reason: The tokeniser splits some words that we would otherwise not think to split (of row 426 of the evaluation data, i.e. after overflow handling):


In [4]:
tokens = ['[CLS]', '[CLS]', 'In', 'welcher', 'Weise', 'wurde', 'die', 'Red', 'Bull', 'Arena', 'in', 'W', '##als', '-', 'Sie', '##zen', '##heim', 'für', 'die', 'Fu', '##ss', '##ball', '-', 'EM', 'umgebaut', '?', '[SEP]', 'der', 'Red', '-', 'Bull', '-', 'Arena', 'werden', 'für', 'der', 'Turnier', 'in', 'zwei', 'Phase', 'um', '##bauen', '.', 'in', 'der', 'erst', 'Phase', 'werden', 'der', 'Stadion', 'für', 'ein', 'Kap', '##azi', '##tät', 'von', '30', '.', '000', 'Zuschauer', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
print("Number of tokens after the [SEP] token:", len(tokens[tokens.index('[SEP]')+1:]))

Number of tokens after the [SEP] token: 229


In [5]:
y="""0    0    0    0    0    0    0    0    0    0    0    1    1    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100""".split()
len(y)

229

In [6]:
yhat="""0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1 1 1
 0 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1
 0 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1""".split()
len(yhat)

229

These are the input tokens after tokenisation. The word „Fußball“ has been split into 3 tokens, at positions 19, 20, 21


In [7]:
print(tokens[19:22])
print(y[19:22])
print(yhat[19:22])

['Fu', '##ss', '##ball']
['0', '0', '0']
['0', '1', '0']


Curiously, they have different labels, according to our model at least. Originally, of course, theyre identical.
Here is another example for Wals-Siezenheim. Wals was apparently false, while Siezenheim was correct. Both town-names were split.

In [8]:
print(tokens[11:17])
print(y[11:17])
print(yhat[11:17])

['W', '##als', '-', 'Sie', '##zen', '##heim']
['1', '1', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '1']


In [31]:
datapath = "../data/output/val_predictions_mbert2.csv"
with open(datapath, "r") as f:
    raw_data = f.readlines()
raw_data = "".join(raw_data).replace("\n", "")[1:-5].split("[END]")
raw_data[13]

"['[CLS]', 'Secondo', 'la', 'leggenda', 'chi', 'sono', 'stati', 'i', 'fondato', '##ri', 'di', 'Lu', '##gd', '##unum', '?', '[SEP]', 'il', 'fondatore', 'di', 'Lu', '##gd', '##unum', 'essere', 'essere', 'il', 'romano', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [30]:
data_lists = [row.split("[SEP]") for row in raw_data]
data_lists[13]

["['[CLS]', 'Secondo', 'la', 'leggenda', 'chi', 'sono', 'stati', 'i', 'fondato', '##ri', 'di', 'Lu', '##gd', '##unum', '?', '",
 "', 'il', 'fondatore', 'di', 'Lu', '##gd', '##unum', 'essere', 'essere', 'il', 'romano', '.', '",
 "', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

each element is now a list of 3 items: query, response, padding, labels, predictions

In [32]:
data_dicts = [{
    "query": row[0].replace("', '", " ").strip().strip("[").strip("]").strip("'").strip("[CLS] "),
    "response": row[1].replace("', '", " ").strip().strip("[").strip("]").strip("'"),
    "labels": [int(lab) for lab in row[3][1:-1].split()],
    "predictions": [int(lab) for lab in row[4][1:-1].split()]
} for row in data_lists]
data_dicts[13]

{'query': 'econdo la leggenda chi sono stati i fondato ##ri di Lu ##gd ##unum ?',
 'response': 'il fondatore di Lu ##gd ##unum essere essere il romano .',
 'labels': [0, 0, 0, 0, 0, 0, 0, 1, 1],
 'predictions': [0, 0, 1, 0, 0, 0, 0, 0, 0]}

Here we have an example of something having gone wrong:
Original text of item 13: I fondatori di Lugdunum sono stati i Romani.
Original hard labels: \[37,43\]
So the model should predict 1 for the tokens "Romani."
But somehow, we have 9 label values and 11 response tokens.

## Solution:
The tokenizer split some words that we would not have split. We need to adjust the texts accordingly to make our analysis possible.

In [15]:
df = pd.DataFrame(data_dicts)
df["query"] = df["query"].apply(lambda x: x.replace(" ##", ""))
df["response"] = df["response"].apply(lambda x: x.replace(" ##", ""))
df

,query,response,labels,predictions
0,"¿ El municipio H ##ü ##ffel ##sheim , está ubi...",h ##ü ##ffel ##sheim ser uno municipio ubicado...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, ..."
1,¿ En qué país se encuentra la ciudad de Tor ##...,el ciudad de to ##ru ##ń él encontrar en el pa...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,¿ En qué estado de Estados Unidos se encuentra...,le ##wis ##town él encontrar en el estado de m...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
3,When did Je ##f Ras ##kin die ?,"Je ##f Ra ##kin die on March 1 , 2011 . he be ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0]","[1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,玳 瑁 蜗 牛 属 的 物 种 分 布 在 哪 些 地 区 ？,玳 瑁 螺 （ 学 名 ： Arch ##acha ##tina margin ##ata ...,"[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
...,...,...,...,...
494,Wie viele Zuschauer haben Platz im Gill ##ette...,der Gil ##ette Stadium sein mit ein Kap ##azi ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, ..."
495,Hu ##r många år har Chris Clare ##mont sk ##ri...,den finn ##as ingen känd upp ##ska ##ttning av...,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
496,金 属 键 决 定 了 金 属 的 哪 些 物 理 性 质 ？,金 属 晶 体 中 ， 金 属 原 子 之 间 通 过 金 属 结 合 力 （ 金 属 离 ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ..."
497,Gegen welche ##n Verein verlor Hamilton Academ...,Hamilton Academic ver ##liere ##n in der Play ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [41]:
with pd.option_context('display.max_colwidth', None):
    display(df.iloc[13])

query          econdo la leggenda chi sono stati i fondatori di Lugdunum ?
response                il fondatore di Lugdunum essere essere il romano .
labels                                         [0, 0, 0, 0, 0, 0, 0, 1, 1]
predictions                                    [0, 0, 1, 0, 0, 0, 0, 0, 0]
accuracy                                                          0.666667
recall                                                                 0.0
Name: 13, dtype: object

In [42]:
df['accuracy'] = df.apply(lambda row: compute_accuracy(row['labels'], row['predictions']), axis=1)
df['recall'] = df.apply(lambda row: compute_recall(row['labels'], row['predictions']), axis=1)
df

,query,response,labels,predictions,accuracy,recall
0,"¿ El municipio Hüffelsheim , está ubicado en e...",hüffelsheim ser uno municipio ubicado en el es...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, ...",0.736842,0.500000
1,¿ En qué país se encuentra la ciudad de Toruń ?,el ciudad de toruń él encontrar en el país de ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.916667,0.000000
2,¿ En qué estado de Estados Unidos se encuentra...,lewistown él encontrar en el estado de montana...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0.666667,0.000000
3,When did Jef Raskin die ?,"Jef Rakin die on March 1 , 2011 . he be 88 yea...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0]","[1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0.600000,0.400000
4,玳 瑁 蜗 牛 属 的 物 种 分 布 在 哪 些 地 区 ？,玳 瑁 螺 （ 学 名 ： Archachatina marginata ） ， 又 称 为...,"[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",0.494118,0.446809
...,...,...,...,...,...,...
494,Wie viele Zuschauer haben Platz im Gillette St...,der Gilette Stadium sein mit ein Kapazität von...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, ...",0.833333,0.857143
495,Hur många år har Chris Claremont skrivit Uncan...,den finnas ingen känd uppskattning av antal si...,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.176471,1.000000
496,金 属 键 决 定 了 金 属 的 哪 些 物 理 性 质 ？,金 属 晶 体 中 ， 金 属 原 子 之 间 通 过 金 属 结 合 力 （ 金 属 离 ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...",0.466667,0.000000
497,Gegen welchen Verein verlor Hamilton Academica...,Hamilton Academic verlieren in der Playoff - F...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.631579,0.000000


In [43]:
#df.apply(lambda row: [token for i, token in enumerate(row["response"].split()) if row["labels"][i] == 1], axis=1)
df.apply(lambda row: (len(row["response"].split()), len(row["labels"])), axis=1)

0       (19, 19)
1       (12, 12)
2       (12, 12)
3       (15, 15)
4      (127, 85)
         ...    
494     (20, 18)
495     (18, 17)
496    (143, 90)
497     (19, 19)
498     (39, 39)
Length: 499, dtype: object

the indices and tokens still dont quite match, investigate further

In [58]:
with pd.option_context('display.max_colwidth', None):
    for i, row in df.iterrows():
        if len(row["response"].split()) != len(row["labels"]):
            print(i)
            print(row["response"])
            print(row["labels"])
            print()

4
玳 瑁 螺 （ 学 名 ： Archachatina marginata ） ， 又 称 为 大 丽 螺 、 宝 螺 ， 是 属 于 腹 足 纲 宝 龟 科 的 一 个 物 种 。 它 主 要 分 布 在 非 洲 的 热 带 地 区 ， 包 括 东 非 、 南 非 、 马 达 加 斯 加 以 及 部 分 西 非 国 家 。 在 这 些 地 区 中 ， 它 可 以 在 各 种 生 境 中 找 到 ， 如 森 林 、 草 原 、 沼 泽 和 农 田 等 。 由 于 它 的 壳 具 有 很 高 的 商 业 价 值 ， 因 此 在 一 些 地 方
[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]

5
चीन ने २०१६ में " Tiantan - 1 " नामक एक उपग्रह लांच करना था । Tianta n - 1 एक संचार उपग्रह है जो उद्देश्य चीन का संचार नेटवर्क को सुदृढ़ करना है । यह उपग्रह २०१६ का ९वाँ माह में लांच करना जाना था ।
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

7
der kazachisch Verein FC Astna - 69 gewinnen sein 2 . Pokal gegen der FC Taraz . < | im _ end | >
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 

Above I print all evaluation results, where somehow the number of tokens used for training and the labels do not match.
This is the case for a variety of languages. For the languages with non-latin characters, I am at a loss. For the others though, I will continue investigating for bugs.

# Ideas to go on:

- Try other tokenisers, e.g. using a language specific one for each observation
- Thus far we havent lemmatized the query before training, maybe that would help
- Try different max_lens
- Try different batch sizes